## American Community Survey 2016 (Millennials and Ethnicity by ZIP)

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import copy
from uszipcode import ZipcodeSearchEngine
from census import Census
from pprint import pprint

# American Community Survey API Key
from config import acs_key

In [2]:
acs5_api_root = "https://api.census.gov/data/2016/acs/acs5?"
acs5_variables = "B19013_001E,\
B01003_001E,\
B01002_001E,\
B01001_002E,\
B01001_010E,\
B01001_011E,\
B01001_012E,\
B01001_026E,\
B01001_034E,\
B01001_035E,\
B01001_036E,\
B02001_001E,\
B02001_002E,\
B02001_003E,\
B02001_004E,\
B02001_005E,\
B02001_006E"

#Query examples: https://api.census.gov/data/2016/acs/acs5/examples.html

#If querying by ZCTA, use the lines below:
geo_unit = "zip%20code%20tabulation%20area:*"
predicates = f"&for={geo_unit}"

#If querying by block group, use the lines below (ignore - using ZIP)
#geo_unit = "block%20group:*"
#state_id = "17"
#county_id = "031"
#tract_id = "*"
#predicates = f"&for={geo_unit}\
#&in=state:{state_id}\
#&in=county:{county_id}\
#&in=tract:{tract_id}"


request_url = f"{acs5_api_root}\
get=NAME,{acs5_variables}\
{predicates}\
&key={acs_key}"

print(request_url)

https://api.census.gov/data/2016/acs/acs5?get=NAME,B19013_001E,B01003_001E,B01002_001E,B01001_002E,B01001_010E,B01001_011E,B01001_012E,B01001_026E,B01001_034E,B01001_035E,B01001_036E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E&for=zip%20code%20tabulation%20area:*&key=86ed69bb313729e12eb61b2d3445f49038b85726


In [3]:
acs5_response = requests.get(request_url)
acs5_json = acs5_response.json()

In [4]:
# Convert to DataFrame
acs5_header = acs5_json[0]
acs5_df = pd.DataFrame(acs5_json[1:], columns=acs5_header)

# Column Reordering
acs5_df = acs5_df.rename(columns={"B19013_001E": "Median HH Income (12 mo)",
                                   "B01003_001E": "Total Population",
                                   "B01002_001E": "Median Age",
                                   "B01001_002E": "Total Pop (M)",
                                   "B01001_010E": "22-24 yo (M)",
                                   "B01001_011E": "25-29 yo (M)",
                                   "B01001_012E": "30-34 yo (M)",
                                   "B01001_026E": "Total Pop (F)",
                                   "B01001_034E": "22-24 yo (F)",
                                   "B01001_035E": "25-29 yo (F)",
                                   "B01001_036E": "30-34 yo (F)",
                                   "B02001_001E": "Total Pop (Race)",
                                   "B02001_002E": "Total Pop (Race-W)",
                                   "B02001_003E": "Total Pop (Race-B)",
                                   "B02001_004E": "Total Pop (Race-N)",
                                   "B02001_005E": "Total Pop (Race-A)",
                                   "B02001_006E": "Total Pop (Race-H)"
                                })

In [5]:
for c in acs5_df.loc[:, 'Median HH Income (12 mo)':'Total Pop (Race-H)']:
    acs5_df[c] = pd.to_numeric(acs5_df[c], errors='ignore')

In [6]:
acs5_df.head()

,NAME,Median HH Income (12 mo),Total Population,Median Age,Total Pop (M),22-24 yo (M),25-29 yo (M),30-34 yo (M),Total Pop (F),22-24 yo (F),25-29 yo (F),30-34 yo (F),Total Pop (Race),Total Pop (Race-W),Total Pop (Race-B),Total Pop (Race-N),Total Pop (Race-A),Total Pop (Race-H),zip code tabulation area
0,ZCTA5 01001,56714,17423,45.0,8059,396,415,583,9364,307,554,619,17423,15974,256,5,703,24,01001
1,ZCTA5 01002,48923,29970,23.2,14536,2104,1019,568,15434,1588,878,615,29970,23459,1711,140,3502,17,01002
2,ZCTA5 01003,2499,11296,19.9,5694,472,70,1,5602,456,9,0,11296,8698,660,30,1538,0,01003
3,ZCTA5 01005,70568,5228,44.1,2798,22,142,158,2430,132,82,47,5228,5063,105,0,32,18,01005
4,ZCTA5 01007,80502,14888,42.5,7224,413,216,323,7664,192,336,327,14888,13906,125,0,443,0,01007


In [7]:
#--------------age--------------#

#combine acs5 age group columns into one millenial group column per sex

    #males who are 22-34 yo
acs5_df['22 - 34 yo (M)'] = acs5_df["22-24 yo (M)"] + acs5_df["25-29 yo (M)"] + acs5_df["30-34 yo (M)"]

    #females who are 22-34 yo
acs5_df['22 - 34 yo (F)'] = acs5_df["22-24 yo (F)"] + acs5_df["25-29 yo (F)"] + acs5_df["30-34 yo (F)"]

    #total population that is 22-34 yo
acs5_df['22 - 34 yo (Total)'] = acs5_df['22 - 34 yo (M)'] + acs5_df['22 - 34 yo (F)']

    #% males who are 22-34 yo
acs5_df['22 - 34 yo (% Pop M)'] = acs5_df['22 - 34 yo (M)']/acs5_df['Total Pop (M)']

    #% females who are 22-34 yo
acs5_df['22 - 34 yo (% Pop F)'] = acs5_df['22 - 34 yo (F)']/acs5_df['Total Pop (F)']

    #% population that is 22-34 yo
acs5_df['22 - 34 yo (% Total)'] = acs5_df['22 - 34 yo (Total)']/acs5_df['Total Population']

    #% population that is 22-34 yo
acs5_df['22 - 34 yo (% Total)'] = acs5_df['22 - 34 yo (Total)']/acs5_df['Total Population']

#--------------race--------------#

    #% population that is White
acs5_df['Race-W (% Total)'] = acs5_df['Total Pop (Race-W)']/acs5_df['Total Population']

    #% population that is Black
acs5_df['Race-B (% Total)'] = acs5_df['Total Pop (Race-B)']/acs5_df['Total Population']

    #% population that is Native American
acs5_df['Race-N (% Total)'] = acs5_df['Total Pop (Race-N)']/acs5_df['Total Population']

    #% population that is Asian
acs5_df['Race-A (% Total)'] = acs5_df['Total Pop (Race-A)']/acs5_df['Total Population']

    #% population that is Native Hawaiian/Pacific Islander 
acs5_df['Race-H (% Total)'] = acs5_df['Total Pop (Race-H)']/acs5_df['Total Population']

acs5_df.head()

,NAME,Median HH Income (12 mo),Total Population,Median Age,Total Pop (M),22-24 yo (M),25-29 yo (M),30-34 yo (M),Total Pop (F),22-24 yo (F),...,22 - 34 yo (F),22 - 34 yo (Total),22 - 34 yo (% Pop M),22 - 34 yo (% Pop F),22 - 34 yo (% Total),Race-W (% Total),Race-B (% Total),Race-N (% Total),Race-A (% Total),Race-H (% Total)
0,ZCTA5 01001,56714,17423,45.0,8059,396,415,583,9364,307,...,1480,2874,0.172974,0.158052,0.164954,0.916834,0.014693,0.000287,0.040349,0.001377
1,ZCTA5 01002,48923,29970,23.2,14536,2104,1019,568,15434,1588,...,3081,6772,0.253921,0.199624,0.225959,0.782749,0.057090,0.004671,0.116850,0.000567
2,ZCTA5 01003,2499,11296,19.9,5694,472,70,1,5602,456,...,465,1008,0.095364,0.083006,0.089235,0.770007,0.058428,0.002656,0.136154,0.000000
3,ZCTA5 01005,70568,5228,44.1,2798,22,142,158,2430,132,...,261,583,0.115082,0.107407,0.111515,0.968439,0.020084,0.000000,0.006121,0.003443
4,ZCTA5 01007,80502,14888,42.5,7224,413,216,323,7664,192,...,855,1807,0.131783,0.111561,0.121373,0.934041,0.008396,0.000000,0.029756,0.000000


## Beggars Delivery ZIP Codes

In [59]:
#get ZIP codes eligible for delivery; examine/clean the ZIP code list
delv_zip = pd.read_csv("Resources\zip_code_raw.csv")
delv_zip_df = pd.DataFrame(delv_zip)

In [60]:
#replace empty strings and white space only values with NaN values
delv_zip_df = delv_zip_df.replace(r'^\s*$', np.nan, regex=True)

#drop rows which have NA values for either StoreID or Zip
delv_zip_df = delv_zip_df[(delv_zip_df.StoreID.notna()) & (delv_zip_df.Zip.notna())]
delv_zip_df.groupby(['StoreID']).count()

#drop duplicate rows (StoreID/Zip combo)
delv_zip_df = delv_zip_df.drop_duplicates(['StoreID','Zip'])
delv_zip_df.groupby(['StoreID'])['Zip'].count()

StoreID
122      45
123       8
10465    21
10584    20
10674    30
10688    18
10766    30
10783    28
11409    44
11467    47
11549    10
11834    27
12086     8
12168    20
12294    27
12340    38
12359    98
12601    50
12896    12
13210    39
13289     6
13604    70
14383    32
Name: Zip, dtype: int64

In [61]:
#reset index: currently each row in the dataframe is a unique StoreID/Zip combo
delv_zip_df = delv_zip_df.reset_index(drop=True)
delv_zip_df.head()

,StoreID,City,State,Zip
0,122,CHICAGO,IL,60655
1,122,BLUE ISLAND,IL,60452
2,122,CHICAGO,IL,60803
3,122,EVERGREEN PARK,NaN,60805
4,122,BLUE ISLAND,IL,60406


In [62]:
#grouping dataframe to get how many stores serve this Zip
#Is this interpretation correct? Does 60452 actually get delivered to by 10 stores?
#what is Zip = 60, 604
group_zip_df = delv_zip_df.groupby(['Zip']).count().sort_values(by='StoreID', ascending=False)
group_zip_df = pd.DataFrame(group_zip_df).reset_index()
group_zip_df.head()

,Zip,StoreID,City,State
0,60452,10,10,10
1,60617,9,9,9
2,60,9,9,8
3,60453,9,9,9
4,60406,9,9,9


In [63]:
#function used to validate values in Zip column.
def valid_zip_test(list):
    
    #build a new list of validated ZIP codes
    valid_zip_list = []
    
    #for every item in the input list
    for item in list:
        #temporary string to store what may or may not be a valid ZIP code
        temp_zip = ''
        
        #for every character in the current item
        for char in item:
            try:
                #if this character is of the integer type, add it to the temp_zip string
                if isinstance(int(char), int):
                    temp_zip += char
            except:
                break
        
        #if the temp_zip string is made of 5 integer-type characters, it's a valid ZIP code. Add it to the valid ZIP list.
        if len(temp_zip) == 5:
            valid_zip_list.append(temp_zip)
    return valid_zip_list

In [64]:
group_zip_list = group_zip_df['Zip'].values.tolist()
group_zip_list[0:5]

['60452', '60617', '60   ', '60453', '60406']

In [65]:
valid_zip_list = valid_zip_test(group_zip_list)
print("The delivery file has " + str(len(valid_zip_list)) + " ZIP codes which are made up of 5 numeric characters.")

The delivery file has 327 ZIP codes which are made up of 5 numeric characters.


In [94]:
valid_zip_list

['60452',
 '60617',
 '60453',
 '60406',
 '60430',
 '60443',
 '60628',
 '60411',
 '60643',
 '60620',
 '60429',
 '60604',
 '60475',
 '60827',
 '60459',
 '60469',
 '60609',
 '60445',
 '60477',
 '60473',
 '60409',
 '60441',
 '60652',
 '60487',
 '60426',
 '46324',
 '60612',
 '60433',
 '60464',
 '60606',
 '60419',
 '60619',
 '60463',
 '60651',
 '60805',
 '60623',
 '60462',
 '60803',
 '60629',
 '60448',
 '60621',
 '60455',
 '60633',
 '60642',
 '60655',
 '46410',
 '60499',
 '60515',
 '60446',
 '60517',
 '46394',
 '60447',
 '60449',
 '60491',
 '60490',
 '60478',
 '60451',
 '60476',
 '60439',
 '60472',
 '60432',
 '60436',
 '60466',
 '60465',
 '60525',
 '46323',
 '46375',
 '46303',
 '60428',
 '60613',
 '60614',
 '60615',
 '60616',
 '60624',
 '60625',
 '60627',
 '60632',
 '60644',
 '60647',
 '60649',
 '60653',
 '60661',
 '60403',
 '60608',
 '60611',
 '60607',
 '46321',
 '60546',
 '60425',
 '60423',
 '60601',
 '60602',
 '60603',
 '46322',
 '60440',
 '46320',
 '46319',
 '46311',
 '60605',
 '46307',


## Add "Beggars Pizza Delivery ZIP Code" Indicator to American Community Survey 2016 DataFrame

In [96]:
#trim down what's in the ACS5 data since some columns are just used for calculations
acs5_df_2 = acs5_df.loc[:,['zip code tabulation area', 'Median HH Income (12 mo)', 'Total Population', 'Median Age', '22 - 34 yo (% Pop M)','22 - 34 yo (% Pop F)','22 - 34 yo (% Total)','Race-W (% Total)', 'Race-B (% Total)', 'Race-N (% Total)', 'Race-A (% Total)', 'Race-H (% Total)']]
acs5_df_2 = acs5_df_2.rename(columns={'zip code tabulation area': 'ZCTA (~ZIP)'})
acs5_df_2.head()

,ZCTA (~ZIP),Median HH Income (12 mo),Total Population,Median Age,22 - 34 yo (% Pop M),22 - 34 yo (% Pop F),22 - 34 yo (% Total),Race-W (% Total),Race-B (% Total),Race-N (% Total),Race-A (% Total),Race-H (% Total)
0,01001,56714,17423,45.0,0.172974,0.158052,0.164954,0.916834,0.014693,0.000287,0.040349,0.001377
1,01002,48923,29970,23.2,0.253921,0.199624,0.225959,0.782749,0.057090,0.004671,0.116850,0.000567
2,01003,2499,11296,19.9,0.095364,0.083006,0.089235,0.770007,0.058428,0.002656,0.136154,0.000000
3,01005,70568,5228,44.1,0.115082,0.107407,0.111515,0.968439,0.020084,0.000000,0.006121,0.003443
4,01007,80502,14888,42.5,0.131783,0.111561,0.121373,0.934041,0.008396,0.000000,0.029756,0.000000


In [103]:
#*-----saving off csv copy of aggregated ACS data first-----*
acs5_df_2.to_csv("Resources\\acs5_2016_census_pop_char_by_zip.csv",index=False)

In [111]:
#have to read the csv as dtype=str or else the leading zeros for ZIP codes drop out
export = pd.read_csv("Resources\\acs5_2016_census_pop_char_by_zip.csv", dtype=str)
export.head()

,ZCTA (~ZIP),Median HH Income (12 mo),Total Population,Median Age,22 - 34 yo (% Pop M),22 - 34 yo (% Pop F),22 - 34 yo (% Total),Race-W (% Total),Race-B (% Total),Race-N (% Total),Race-A (% Total),Race-H (% Total)
0,01001,56714,17423,45.0,0.17297431443107086,0.15805211448099102,0.1649543706594731,0.9168340699075934,0.014693221603627389,0.0002869769844458474,0.04034896401308615,0.0013774895253400676
1,01002,48923,29970,23.2,0.25392129884424874,0.1996242062977841,0.2259592926259593,0.7827494160827494,0.057090423757090426,0.004671338004671338,0.11685018351685018,0.000567233900567234
2,01003,2499,11296,19.9,0.09536354056902002,0.08300606926097823,0.08923512747875353,0.7700070821529745,0.058427762039660054,0.0026558073654390935,0.1361543909348442,0.0
3,01005,70568,5228,44.1,0.11508220157255182,0.10740740740740741,0.11151491966335118,0.9684391736801836,0.02008416220351951,0.0,0.00612088752869166,0.0034429992348890587
4,01007,80502,14888,42.5,0.13178294573643412,0.11156054279749478,0.12137291778613649,0.9340408382590005,0.008396023643202579,0.0,0.02975550779150994,0.0


## *------- the analysis below isn't working as expected: "is in"?--------*

In [16]:
#add indicator column - does Beggars Pizza deliver to this ACS ZIP?
acs5_df_2['BP Delivers'] = acs5_df_2['ZCTA (~ZIP)'].isin(valid_zip_list)

In [17]:
acs5_df_2.head()

,ZCTA (~ZIP),Median HH Income (12 mo),Total Population,Median Age,22 - 34 yo (% Pop M),22 - 34 yo (% Pop F),22 - 34 yo (% Total),Race-W (% Total),Race-B (% Total),Race-N (% Total),Race-A (% Total),Race-H (% Total),BP Delivers
0,01001,56714,17423,45.0,0.172974,0.158052,0.164954,0.916834,0.014693,0.000287,0.040349,0.001377,False
1,01002,48923,29970,23.2,0.253921,0.199624,0.225959,0.782749,0.057090,0.004671,0.116850,0.000567,False
2,01003,2499,11296,19.9,0.095364,0.083006,0.089235,0.770007,0.058428,0.002656,0.136154,0.000000,False
3,01005,70568,5228,44.1,0.115082,0.107407,0.111515,0.968439,0.020084,0.000000,0.006121,0.003443,False
4,01007,80502,14888,42.5,0.131783,0.111561,0.121373,0.934041,0.008396,0.000000,0.029756,0.000000,False


In [18]:
#dataframe that only has rows for ZIP codes from ACS 2016 data which Beggars Pizza delivers to
acs5_df_3a = acs5_df_2.loc[acs5_df_2['BP Delivers'] == True]
acs5_df_3a = acs5_df_3a.reset_index(drop=True)
len(acs5_df_3a)

268

In [20]:
acs5_df_3 = acs5_df_3a[(acs5_df_3a['Median HH Income (12 mo)'] != -666666666) & (acs5_df_3a['Total Population'] != 0)]
len(acs5_df_3)

265

## Store Locations (this part works)

In [21]:
#get ZIP codes eligible for delivery; examine/clean the ZIP code list
store_locs = pd.read_csv("Resources\projectbeggarsstart_geo.csv")
store_locs_df = pd.DataFrame(store_locs)

In [22]:
#for each store, use its latitude and longitude values to find its ZIP code
search = ZipcodeSearchEngine()
store_ZIPs = []
for row_index, row in store_locs_df.iterrows():
    zipcode = search.by_coordinate(row.Latitude,row.Longitude, returns=1)
    store_ZIPs.append(zipcode[0]['Zipcode'])
store_locs_df['ZIP'] = store_ZIPs
store_locs_df.head()

,SQL_ID,Address,City,State,Latitude,Longitude,ZIP
0,122,12700 S Kedzie,Blue Island,IL,41.661707,-87.700403,60406
1,123,12660 S Western,Blue Island,IL,41.663078,-87.680465,60406
2,10465,3277 W 115th St,Merrionette Park,IL,41.683913,-87.701136,60655
3,11834,22149 Governors Hwy,Richton Park,IL,41.485875,-87.712412,60471
4,10674,3524 Ridge Road,Lansing,IL,41.564555,-87.531317,60438


### Add StoreID to ACS dataframe (acs5_df_3, ZIP codes which are delivered to)
*can't assume that a ZIP code is delivered exclusively to by one store; i.e. multiple stores can deliver to the same ZIP code

In [32]:
#acs5_df_3 limited to BP delivery ZIPs 
acs5_df_3 = acs5_df_2b.loc[acs5_df_2b['BP Delivers'] == True]
acs5_df_3 = acs5_df_3.reset_index(drop=True)
len(acs5_df_3)

268

In [66]:
#make unique ID for each StoreID_Zip combo in delivery file
delv_zip_df_2 = delv_zip_df
delv_zip_df_2['StoreID'] = delv_zip_df_2['StoreID'].astype(str)
delv_zip_df_2['StoreID_Zip'] = delv_zip_df_2['StoreID'] + "_" + delv_zip_df_2['Zip']
delv_zip_df_2.head()
len(delv_zip_df_2)

728

In [93]:
#making store column for just the ACS ZIP codes that matched "BP Delivers" == True (268)
#adding boolean value if this store delivers to this ZIP code

In [35]:
#--------there's something in here that isn't working the way I think it is----------#
#for each acs5_df_3 column from 122 to the end
for c_index in acs5_df_3.loc[:, '122':'14581']:
    
    #for each row in ACS5_df_3, put True if there's a match in delv_zip_df ZCTA (~ZIP)' and storeID both match 
    for r_index, row in acs5_df_3.iterrows():
        StoreID_Zip = c_index + "_" + acs5_df_3['ZCTA (~ZIP)']
        acs5_df_3[c_index] = delv_zip_df_2['StoreID_Zip'].isin(c_index + "_" + acs5_df_3['ZCTA (~ZIP)'])

In [49]:
#acs5_df_4 is a dataframe where each row corresponds with 1 ZIP code.
#The ZIP codes in this dataframe are limited to the ones which Beggars Pizza delivers to.
#The delivery ZIP code list has more entries than acs5_df_3
#difference may be because those ZIPs no longer exist or there is some funny data entry
acs5_df_4 = acs5_df_3
acs5_df_4

,ZCTA (~ZIP),Median HH Income (12 mo),Total Population,Median Age,22 - 34 yo (% Pop M),22 - 34 yo (% Pop F),22 - 34 yo (% Total),Race-W (% Total),Race-B (% Total),Race-N (% Total),...,11549,12168,12359,12601,13289,13210,11409,10584,14593,14581
0,02109,117344,4015,32.9,0.474126,0.433295,0.456289,0.870984,0.033624,0.000000,...,False,False,False,False,False,False,False,False,False,False
1,02130,79419,39316,33.9,0.329697,0.324037,0.326610,0.654136,0.119442,0.002416,...,False,False,False,False,False,False,False,False,False,False
2,02210,155313,2461,35.5,0.429275,0.419231,0.425030,0.889882,0.034945,0.000000,...,False,False,False,False,False,False,False,False,False,False
3,07054,83936,29555,40.4,0.197418,0.197374,0.197395,0.541803,0.049332,0.002504,...,False,False,False,False,False,False,False,False,False,False
4,07920,132417,26949,44.0,0.081701,0.087157,0.084604,0.778322,0.016810,0.000000,...,False,False,False,False,False,False,False,False,False,False
5,07974,145607,12110,40.1,0.111888,0.110092,0.110983,0.851858,0.007432,0.002973,...,False,False,False,False,False,False,False,False,False,False
6,08536,95533,20180,34.8,0.232041,0.219203,0.225322,0.333944,0.070466,0.009613,...,False,False,False,False,False,False,False,False,False,False
7,15238,84038,13329,49.8,0.095298,0.097848,0.096631,0.876510,0.012529,0.000000,...,False,False,False,False,False,False,False,False,False,False
8,19067,100455,51628,43.1,0.145831,0.133045,0.139362,0.838247,0.069865,0.000407,...,False,False,False,False,False,False,False,False,False,False
9,19112,-666666666,0,-666666666.0,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [55]:
for c_index in acs5_df_4.loc[:, '122':'14581']:
    print("Store " + c_index + " delivers to " + str(np.sum(acs5_df_3[c_index])) + " ACS ZIP codes.") 

Store 122 delivers to 28 ACS ZIP codes.
Store 123 delivers to 2 ACS ZIP codes.
Store 10465 delivers to 11 ACS ZIP codes.
Store 11834 delivers to 15 ACS ZIP codes.
Store 10674 delivers to 27 ACS ZIP codes.
Store 10766 delivers to 19 ACS ZIP codes.
Store 10783 delivers to 0 ACS ZIP codes.
Store 12294 delivers to 15 ACS ZIP codes.
Store 12340 delivers to 25 ACS ZIP codes.
Store 12896 delivers to 6 ACS ZIP codes.
Store 14383 delivers to 0 ACS ZIP codes.
Store 12086 delivers to 4 ACS ZIP codes.
Store 13604 delivers to 0 ACS ZIP codes.
Store 10688 delivers to 16 ACS ZIP codes.
Store 11467 delivers to 12 ACS ZIP codes.
Store 11549 delivers to 4 ACS ZIP codes.
Store 12168 delivers to 11 ACS ZIP codes.
Store 12359 delivers to 23 ACS ZIP codes.
Store 12601 delivers to 19 ACS ZIP codes.
Store 13289 delivers to 0 ACS ZIP codes.
Store 13210 delivers to 13 ACS ZIP codes.
Store 11409 delivers to 0 ACS ZIP codes.
Store 10584 delivers to 0 ACS ZIP codes.
Store 14593 delivers to 0 ACS ZIP codes.
Store 1

In [68]:
#testing ZIP match
delv_zip_df_3 = delv_zip_df
delv_zip_df_3.head()

,StoreID,City,State,Zip,StoreID_Zip
0,122,CHICAGO,IL,60655,122_60655
1,122,BLUE ISLAND,IL,60452,122_60452
2,122,CHICAGO,IL,60803,122_60803
3,122,EVERGREEN PARK,NaN,60805,122_60805
4,122,BLUE ISLAND,IL,60406,122_60406


In [90]:
delv_zip_of_store = delv_zip_df_3['Zip'].loc[delv_zip_df_3['StoreID'] == '14383']
delv_zip_of_store_list = delv_zip_df_3['Zip'].loc[delv_zip_df_3['StoreID'] == '14383'].tolist()

In [92]:
np.sum(delv_zip_of_store.isin(valid_zip_list_acs))

30

In [89]:
np.sum(valid_zip_list_acs.isin(delv_zip_of_store_list))

30

In [37]:
valid_zip_list_acs = acs5_df_4['ZCTA (~ZIP)']
valid_zip_list_acs

0      02109
1      02130
2      02210
3      07054
4      07920
5      07974
6      08536
7      15238
8      19067
9      19112
10     21028
11     21901
12     22193
13     27377
14     27703
15     28202
16     30144
17     43612
18     46017
19     46033
20     46221
21     46303
22     46307
23     46310
24     46311
25     46312
26     46319
27     46320
28     46321
29     46322
       ...  
238    60649
239    60651
240    60652
241    60653
242    60654
243    60655
244    60657
245    60659
246    60661
247    60707
248    60803
249    60804
250    60805
251    60827
252    60901
253    60914
254    60915
255    60950
256    60951
257    60964
258    61054
259    61516
260    62324
261    62411
262    63011
263    63104
264    78735
265    90057
266    90061
267    91011
Name: ZCTA (~ZIP), Length: 268, dtype: object

In [38]:
store_list = acs5_df_4.loc[:, '122':'14581'].columns.values.tolist()
store_list

['122',
 '123',
 '10465',
 '11834',
 '10674',
 '10766',
 '10783',
 '12294',
 '12340',
 '12896',
 '14383',
 '12086',
 '13604',
 '10688',
 '11467',
 '11549',
 '12168',
 '12359',
 '12601',
 '13289',
 '13210',
 '11409',
 '10584',
 '14593',
 '14581']

In [40]:
#https://stackoverflow.com/questions/40973687/create-new-dataframe-in-pandas-with-dynamic-names-also-add-new-column
store_df_dict = {}

for store in store_list:

    store_df_name = "acs5_df_4_store_" + store
    store_df = acs5_df_4.loc[acs5_df_4[store] == True]
    store_df_dict[store_df_name] = store_df

store_df_dict

{'acs5_df_4_store_122':     ZCTA (~ZIP)  Median HH Income (12 mo)  Total Population   Median Age  \
 0         02109                    117344              4015         32.9   
 1         02130                     79419             39316         33.9   
 2         02210                    155313              2461         35.5   
 3         07054                     83936             29555         40.4   
 4         07920                    132417             26949         44.0   
 5         07974                    145607             12110         40.1   
 6         08536                     95533             20180         34.8   
 7         15238                     84038             13329         49.8   
 8         19067                    100455             51628         43.1   
 9         19112                -666666666                 0 -666666666.0   
 10        21028                    105234              3043         48.1   
 11        21901                     70983           

In [43]:
#example of one store's ZIP code info
store_df_dict['acs5_df_4_store_10766']

,ZCTA (~ZIP),Median HH Income (12 mo),Total Population,Median Age,22 - 34 yo (% Pop M),22 - 34 yo (% Pop F),22 - 34 yo (% Total),Race-W (% Total),Race-B (% Total),Race-N (% Total),...,11549,12168,12359,12601,13289,13210,11409,10584,14593,14581
82,60164,60000,21700,36.0,0.180959,0.167942,0.174608,0.716267,0.032995,0.010230,...,False,False,False,False,False,False,False,False,False,False
83,60165,40717,4936,30.8,0.189817,0.227731,0.208874,0.533023,0.020259,0.000000,...,False,False,False,False,False,False,False,False,False,False
84,60175,126281,26334,40.6,0.127300,0.102664,0.115782,0.936318,0.014278,0.002392,...,False,False,False,False,False,False,False,False,False,False
85,60181,70000,28667,37.6,0.184760,0.190108,0.187393,0.768549,0.056267,0.002686,...,False,False,False,False,False,False,False,False,False,False
86,60189,93222,31559,40.4,0.152676,0.166575,0.159859,0.862163,0.052093,0.000222,...,False,False,False,False,False,False,False,False,False,False
87,60201,70497,43056,30.8,0.204205,0.158388,0.180254,0.685619,0.132757,0.001185,...,False,False,False,False,False,False,False,False,False,False
88,60301,70040,2329,45.2,0.261287,0.191770,0.222842,0.604122,0.261915,0.000000,...,False,False,False,False,False,False,False,False,False,False
89,60302,79167,32258,39.6,0.177817,0.171934,0.174747,0.657573,0.221216,0.000589,...,False,False,False,False,False,False,False,False,False,False
90,60304,93903,17402,37.7,0.159742,0.157404,0.158488,0.702505,0.186013,0.000517,...,False,False,False,False,False,False,False,False,False,False
91,60305,103622,11217,40.7,0.080138,0.100805,0.090666,0.818133,0.067398,0.000000,...,False,False,False,False,False,False,False,False,False,False


### Population Characteristics by Store (Weighted)

In [44]:
#http://pbpython.com/weighted-average.html
def weighted_average(dataframe, avg_name, weight_name):
    d = dataframe[avg_name]
    w = dataframe[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [46]:
characteristics_to_weight = acs5_df_4.loc[:, 'Median HH Income (12 mo)':'Race-H (% Total)'].columns.values.tolist()
characteristics_to_weight

['Median HH Income (12 mo)',
 'Total Population',
 'Median Age',
 '22 - 34 yo (% Pop M)',
 '22 - 34 yo (% Pop F)',
 '22 - 34 yo (% Total)',
 'Race-W (% Total)',
 'Race-B (% Total)',
 'Race-N (% Total)',
 'Race-A (% Total)',
 'Race-H (% Total)']

In [47]:
store_df_dict_keys = store_df_dict.keys()
store_df_dict_keys = list((store_df_dict_keys))
store_df_dict_keys

['acs5_df_4_store_122',
 'acs5_df_4_store_123',
 'acs5_df_4_store_10465',
 'acs5_df_4_store_11834',
 'acs5_df_4_store_10674',
 'acs5_df_4_store_10766',
 'acs5_df_4_store_10783',
 'acs5_df_4_store_12294',
 'acs5_df_4_store_12340',
 'acs5_df_4_store_12896',
 'acs5_df_4_store_14383',
 'acs5_df_4_store_12086',
 'acs5_df_4_store_13604',
 'acs5_df_4_store_10688',
 'acs5_df_4_store_11467',
 'acs5_df_4_store_11549',
 'acs5_df_4_store_12168',
 'acs5_df_4_store_12359',
 'acs5_df_4_store_12601',
 'acs5_df_4_store_13289',
 'acs5_df_4_store_13210',
 'acs5_df_4_store_11409',
 'acs5_df_4_store_10584',
 'acs5_df_4_store_14593',
 'acs5_df_4_store_14581']

In [48]:
for store_df_key in store_df_dict_keys:
    print(str(store_df_key) + " has " + str(store_df_dict[store_df_key]['ZCTA (~ZIP)'].count()) + " ZIPs")

acs5_df_4_store_122 has 28 ZIPs
acs5_df_4_store_123 has 2 ZIPs
acs5_df_4_store_10465 has 11 ZIPs
acs5_df_4_store_11834 has 15 ZIPs
acs5_df_4_store_10674 has 27 ZIPs
acs5_df_4_store_10766 has 19 ZIPs
acs5_df_4_store_10783 has 0 ZIPs
acs5_df_4_store_12294 has 15 ZIPs
acs5_df_4_store_12340 has 25 ZIPs
acs5_df_4_store_12896 has 6 ZIPs
acs5_df_4_store_14383 has 0 ZIPs
acs5_df_4_store_12086 has 4 ZIPs
acs5_df_4_store_13604 has 0 ZIPs
acs5_df_4_store_10688 has 16 ZIPs
acs5_df_4_store_11467 has 12 ZIPs
acs5_df_4_store_11549 has 4 ZIPs
acs5_df_4_store_12168 has 11 ZIPs
acs5_df_4_store_12359 has 23 ZIPs
acs5_df_4_store_12601 has 19 ZIPs
acs5_df_4_store_13289 has 0 ZIPs
acs5_df_4_store_13210 has 13 ZIPs
acs5_df_4_store_11409 has 0 ZIPs
acs5_df_4_store_10584 has 0 ZIPs
acs5_df_4_store_14593 has 0 ZIPs
acs5_df_4_store_14581 has 0 ZIPs
